In [24]:
import networkx as nx
import matplotlib.pyplot as plt
import scipy as sci
import numpy as np
import numpy.linalg as linalg
import numpy.random as rand
import pandas as pd
from platform import node
from cdlib.classes.node_clustering import NodeClustering
import community.community_louvain as community_louvain
import cdlib as cd
from collections import defaultdict
import networkx as nx
import cdlib.algorithms as algo
import cdlib.viz as viz
import matplotlib.pyplot as plt

from functions import louvain

In [25]:
hero_edges = pd.read_csv('hero-network.csv')

hero_graph = nx.from_pandas_edgelist(hero_edges, 'hero1', 'hero2', create_using=nx.Graph())

In [28]:
def louvain(
    g_original: object,
    weight: str = "weight",
    resolution: float = 1.0,
    randomize: int = None) -> NodeClustering:
    """
    Louvain  maximizes a modularity score for each community.
    The algorithm optimises the modularity in two elementary phases:
    (1) local moving of nodes;
    (2) aggregation of the network.
    In the local moving phase, individual nodes are moved to the community that yields the largest increase in the quality function.
    In the aggregation phase, an aggregate network is created based on the partition obtained in the local moving phase.
    Each community in this partition becomes a node in the aggregate network. The two phases are repeated until the quality function cannot be increased further.


    **Supported Graph Types**

    ========== ======== ========
    Undirected Directed Weighted
    ========== ======== ========
    Yes        No       No
    ========== ======== ========

    :param g_original: a networkx/igraph object
    :param weight: str, optional the key in graph to use as weight. Default to 'weight'
    :param resolution: double, optional  Will change the size of the communities, default to 1.
    :param randomize: int, RandomState instance or None, optional (default=None). If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by `np.random`.
    :return: NodeClustering object


    :Example:

    >>> from cdlib import algorithms
    >>> import networkx as nx
    >>> G = nx.karate_club_graph()
    >>> coms = algorithms.louvain(G, weight='weight', resolution=1.)

    :References:

    Blondel, Vincent D., et al. `Fast unfolding of communities in large networks. <https://iopscience.iop.org/article/10.1088/1742-5468/2008/10/P10008/meta/>`_ Journal of statistical mechanics: theory and experiment 2008.10 (2008): P10008.

    .. note:: Reference implementation: https://github.com/taynaud/python-louvain
    """
    g = g_original
    coms = community_louvain.best_partition(
        g, weight=weight, resolution=resolution, randomize=randomize
    )

    # Reshaping the results
    coms_to_node = defaultdict(list)
    for n, c in coms.items():
        coms_to_node[c].append(n)

    coms_louvain = [list(c) for c in coms_to_node.values()]
    return cd.NodeClustering(
        coms_louvain,
        g_original,
        "Louvain",
        method_parameters={
            "weight": weight,
            "resolution": resolution,
            "randomize": randomize,
        },
    )

In [29]:
MarvelCluster = louvain(hero_graph)

In [30]:
print(len(MarvelCluster.communities))

25


In [41]:
coms = np.array(MarvelCluster.communities)
coms.shape()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (25,) + inhomogeneous part.

In [36]:
coms = pd.DataFrame(MarvelCluster.communities)
coms.head()

,0,1,2,3,4,5,6,7,8,9,...,1337,1338,1339,1340,1341,1342,1343,1344,1345,1346
0,MAN-THING/THEODORE T,MYSTIQUE/RAVEN DARKH,BLOB/FRED J. DUKES,PYRO/ALLERDYCE JOHNN,AVALANCHE/DOMINIC PE,ROGUE /,DESTINY II/IRENE ADL,HYBRID/JAMES JIMMY M,WOLVERINE/LOGAN,PSYLOCKE/ELISABETH B,...,"WENTWORTH, JENNIFER","BLOTT, BRIGADIER INK",SA'TNEEN,BURNING PUDDLE,"KANE, SUGAR",MISTER CLEAN,MISS SACCHARINE,CYCLOPS II,HARMONY,WARP SAVANT
1,PRINCESS ZANDA,"STEELE, SIMON/WOLFGA","FORTUNE, DOMINIC","RAVEN, SABBATH II/EL",CARNIVORE/COUNT ANDR,SPIDER-MAN/PETER PAR,PUNISHER II/FRANK CA,RHINO,"STACY, PAUL",VENOM/EDDIE BROCK,...,None,None,None,None,None,None,None,None,None,None
2,GALACTUS/GALAN,SILVER SURFER/NORRIN,LORD CHAOS,SCARLET WITCH/WANDA,UATU,THUNDERSTRIKE/ERIC K,WONDER MAN/SIMON WIL,DEATHLOK III/MICHAEL,OBLIVION,SHE-HULK/JENNIFER WA,...,None,None,None,None,None,None,None,None,None,None
3,"LITTLE, ABNER","ERWIN, CLYTEMNESTRA",IRON MAN/TONY STARK,IRON MAN IV/JAMES R.,GHOST,"ZIMMER, ABE",HAWK,CAPTAIN AMERICA,LIVING LASER/ARTHUR,"CABE, BETHANY",...,None,None,None,None,None,None,None,None,None,None
4,BLACK PANTHER/T'CHAL,"ROM, SPACEKNIGHT",TORPEDO III/BROCK JO,SUB-MARINER/NAMOR MA,VASHTI,SEAWEED MAN,HULK/DR. ROBERT BRUC,TIGRA/GREER NELSON,WASP/JANET VAN DYNE,MOCKINGBIRD/DR. BARB,...,None,None,None,None,None,None,None,None,None,None
